In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

img_size = 299
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while(True):
    ret, frame = cam.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for (x,y,w,h) in faces:
        if w >130:
            detected_face = frame[int(y):int(y+h), int(x):int(x+w)] 
            cv2.imshow("Test Photo", detected_face)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Hit esc to skip and hit the spacebar to save image")
        break
    elif k%256 == 32:
        # SPACE pressed 
        faceresize = cv2.resize(detected_face, (img_size,img_size)) 
        img_name = "data/via/opencv_frame_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, faceresize)
        print("{} written!".format(img_name))
        img_counter += 1
    
cam.release()

cv2.destroyAllWindows()

data/via/opencv_frame_0.jpg written!
data/via/opencv_frame_1.jpg written!
data/via/opencv_frame_2.jpg written!
data/via/opencv_frame_3.jpg written!
data/via/opencv_frame_4.jpg written!
data/via/opencv_frame_5.jpg written!
data/via/opencv_frame_6.jpg written!
data/via/opencv_frame_7.jpg written!
data/via/opencv_frame_8.jpg written!
data/via/opencv_frame_9.jpg written!
data/via/opencv_frame_10.jpg written!
data/via/opencv_frame_11.jpg written!
data/via/opencv_frame_12.jpg written!
data/via/opencv_frame_13.jpg written!
data/via/opencv_frame_14.jpg written!
data/via/opencv_frame_15.jpg written!
data/via/opencv_frame_16.jpg written!
data/via/opencv_frame_17.jpg written!
data/via/opencv_frame_18.jpg written!
data/via/opencv_frame_19.jpg written!
data/via/opencv_frame_20.jpg written!
data/via/opencv_frame_21.jpg written!
data/via/opencv_frame_22.jpg written!
data/via/opencv_frame_23.jpg written!
data/via/opencv_frame_24.jpg written!
data/via/opencv_frame_25.jpg written!
data/via/opencv_frame_

In [4]:
train_path = 'data/trainimgface.csv'
test_path = 'data/testimgface.csv'
train_data = pd.read_csv(train_path)  
test_data = pd.read_csv(test_path)

In [14]:
train_data.head()

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,...,11y,12x,12y,13x,13y,14x,14y,15x,15y,image
0,130,127,146,126,156,127,193,145,203,148,...,181,195,217,164,247,111,209,78,182,trainimg_1.jpg
1,108,163,114,155,119,151,145,131,152,123,...,162,220,209,169,248,131,224,120,195,trainimg_2.jpg
2,103,135,115,131,126,133,166,131,178,126,...,164,214,200,162,237,108,214,96,176,trainimg_3.jpg
3,98,137,107,133,122,129,162,131,174,125,...,166,204,218,155,254,101,220,90,185,trainimg_4.jpg
4,103,145,113,135,123,135,155,129,164,120,...,175,224,210,170,251,122,215,111,181,trainimg_5.jpg


In [15]:
train_data.shape[1]

33

In [16]:
test_data.head()

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,...,11y,12x,12y,13x,13y,14x,14y,15x,15y,image
0,117,128,127,126,133,123,149,122,157,117,...,160,175,187,158,205,130,192,119,170,testimg_1.jpg
1,87,71,109,71,123,75,180,75,193,70,...,126,213,152,148,170,106,158,71,119,testimg_2.jpg
2,123,154,138,150,156,150,197,155,206,154,...,193,203,217,170,228,133,218,104,190,testimg_3.jpg
3,116,121,123,117,132,114,152,111,164,101,...,160,184,186,152,219,127,204,118,172,testimg_4.jpg
4,87,145,94,141,103,137,140,135,152,131,...,172,182,192,149,214,111,207,91,184,testimg_5.jpg


In [19]:
coltrn = train_data['image']
imgs = []
print (coltrn.shape[0])
training = train_data.drop('image',axis = 1)
Y_train = []
for i in range (coltrn.shape[0]):
    p = os.path.join(os.getcwd(), 'data/'+str(coltrn.iloc[i]))
    print(p)
    img = cv2.imread(p, 1)

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgs.append(gray_img)
    
    y = training.iloc[i,:]
    Y_train.append(y)

X_train = np.asarray(imgs)

Y_train = np.array(Y_train,dtype = 'float')  

print(X_train.shape, Y_train.shape)

50
/Users/drewperkins/Documents/cv/hashing_filtering_detection/key_point_prediction/data/trainimg_1.jpg


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [5]:
coltst = test_data['image']
imgstst = []
print (coltst.shape[0])
testing = test_data.drop('image',axis = 1)
Y_test = []
for i in range (coltst.shape[0]):
    p = os.path.join(os.getcwd(), 'faceimagestrain/'+str(coltst.iloc[i]))
    imgtst = cv2.imread(p, 1)
    gray_imgtst = cv2.cvtColor(imgtst, cv2.COLOR_BGR2GRAY)
    imgstst.append(gray_imgtst)
    
    y = testing.iloc[i,:]
    Y_test.append(y)

X_test = np.asarray(imgstst)

Y_test = np.array(Y_test,dtype = 'float')  

print(X_test.shape, Y_test.shape)

7


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
Y_trainx = training.loc[:,['0x','1x','2x','3x','4x','5x','6x','7x','8x','9x','10x','11x','12x','13x','14x','15x']] 
Y_trainy = training.loc[:,['0y','1y','2y','3y','4y','5y','6y','7y','8y','9y','10y','11y','12y','13y','14y','15y']] 

print(Y_trainx.shape, Y_trainy.shape)
Y_trainx.head()

NameError: name 'training' is not defined